In [1]:
# -*- coding: utf-8 -*-
## 띄어쓰기때문에 spark-warehouse경로 명시적으로 설정

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL 저장하기") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.sql.adaptive.enabled", False) \
    .enableHiveSupport() \
    .getOrCreate()

In [2]:
url = "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234"

In [3]:
df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

In [4]:
df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()

In [5]:
join_expr = df_user_session_channel.sessionid == df_session_timestamp.sessionid
df_join = df_user_session_channel.join(df_session_timestamp, join_expr, "inner")
df_join.show(10)

+------+--------------------+--------+--------------------+--------------------+
|userid|           sessionid| channel|           sessionid|                  ts|
+------+--------------------+--------+--------------------+--------------------+
|  1501|0135456d6a3c1051f...|  Google|0135456d6a3c1051f...|2019-09-24 14:49:...|
|   876|01a416a7e28d0d229...|Facebook|01a416a7e28d0d229...|2019-05-26 14:23:...|
|   243|0226aa5193c66d990...|  Google|0226aa5193c66d990...|2019-07-01 23:04:...|
|  2776|029bf49b584c641f0...|Facebook|029bf49b584c641f0...|2019-11-11 20:37:...|
|   939|02b8d6c2775b756de...|  Google|02b8d6c2775b756de...|2019-09-01 15:29:...|
|  2133|02ea66ee24d57e285...| Organic|02ea66ee24d57e285...| 2019-10-28 22:50:40|
|   771|02f6364bf207d6837...|   Naver|02f6364bf207d6837...|2019-09-07 18:24:...|
|  1961|0302915889fa38fe5...| Youtube|0302915889fa38fe5...| 2019-11-29 15:16:49|
|  2139|039cc47f960921e3c...|   Naver|039cc47f960921e3c...| 2019-08-01 18:19:34|
|   599|03ce0331cbd983f16...

In [6]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [20]:
spark.sql("DROP TABLE IF EXISTS bk_usc")
spark.sql("DROP TABLE IF EXISTS bk_st")

DataFrame[]

In [7]:
df_user_session_channel.write.bucketBy(3, "sessionid").saveAsTable("bk_usc",mode="overwrite")
df_session_timestamp.write.bucketBy(3, "sessionid").saveAsTable("bk_st",mode="overwrite")

In [9]:
print(spark.conf.get("spark.sql.warehouse.dir"))

file:/C:/Users/Yeojun/Documents/GitHub/Programmers_DevCourse/학습내용/빅데이터%20처리%20시스템%20하둡과%20Spark/실습/Spark내부동작/spark-warehouse


In [16]:
df_bk_usc = spark.read.table("bk_usc")
df_bk_st = spark.read.table("bk_st")

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `bk_usc` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [bk_usc], [], false
